In [1]:
# Importing all the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Navigating to Chest-X-Ray Folder.
Data_Path= "/content/drive/My Drive/Chest_Xray"

In [4]:
# Getting Paths of Training, Testing, Labels files from Drive.
import os

Train_Folder = os.path.join(Data_Path, 'train_images')
Labels_csv = os.path.join(Data_Path, 'labels_train.csv')

In [5]:
# Loading the Labels
Labels=pd.read_csv(Labels_csv)
Labels.head(8)

,file_name,class_id
0,img_4358977458434011046.jpg,1
1,img_5224016757187192130.jpg,2
2,img_3065202206106254707.jpg,2
3,img_6304894865561547174.jpg,1
4,img_3371338542810939877.jpg,2
5,img_946797649386887230.jpg,0
6,img_3282952127033205295.jpg,0
7,img_3090153597350897926.jpg,1


In [6]:
# Labels:
# 0-Normal
# 1-Bacterial_Pneumonia
# 2-Viral_Pneumonia

In [7]:
Labels.info()
Labels['class_id']=Labels['class_id'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4672 entries, 0 to 4671
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  4672 non-null   object
 1   class_id   4672 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 73.1+ KB


In [8]:
# Add a column for full image paths
Labels['path'] = Labels['file_name'].apply(lambda x: os.path.join(Train_Folder, x))
print(Labels['path'].head())

0    /content/drive/My Drive/Chest_Xray/train_image...
1    /content/drive/My Drive/Chest_Xray/train_image...
2    /content/drive/My Drive/Chest_Xray/train_image...
3    /content/drive/My Drive/Chest_Xray/train_image...
4    /content/drive/My Drive/Chest_Xray/train_image...
Name: path, dtype: object


In [9]:
# Since the Donot have Validation Dataset we are going to Split the training dataset into training and validation sets.

from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(Labels, test_size=0.2, random_state=42,stratify=Labels['class_id'])

#stratify: Maintains class distribution across training and validation sets.

print(f"Training data: {train_data.shape}")
print(f"Validation data: {val_data.shape}")

Training data: (3737, 3)
Validation data: (935, 3)


In [10]:
# Use ImageDataGenerator to preprocess and augment the images dynamically during training:

Train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only Normalising for Validation Data
val_datagen = ImageDataGenerator(rescale=1.0/255)

In [11]:
# Training data generator
train_generator = Train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='path',
    y_col='class_id',
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,
    class_mode='sparse', # Multi-class classification
    repeat=True
)

# Validation data generator

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    x_col='path',
    y_col='class_id',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

Found 3737 validated image filenames belonging to 3 classes.
Found 935 validated image filenames belonging to 3 classes.


In [12]:
# Define a CNN Model

X_Ray_Model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),  # First convolutional layer

    MaxPooling2D(pool_size=(2, 2)),  # First pooling layer

    Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer

    MaxPooling2D(pool_size=(2, 2)),  # Second pooling layer

    Flatten(),  # Flatten the feature maps

    Dense(128, activation='relu'),  # Fully connected layer

    Dropout(0.5),  # Dropout to prevent overfitting

    Dense(3, activation='softmax')  # Output layer for 3 classes
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compile the model with an optimizer, loss function, and evaluation metric:

X_Ray_Model.compile(
    optimizer='adam',
    loss='SparseCategoricalCrossentropy',
    metrics=['accuracy']
)

In [14]:
# Training the Model

for epochs in range(20):
  Training = X_Ray_Model.fit(train_generator,validation_data=val_generator,epochs=1,steps_per_epoch=len(train_data)//32,validation_steps=len(val_generator))
  train_generator.reset()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


116/116 ━━━━━━━━━━━━━━━━━━━━ 1299s 11s/step - accuracy: 0.4582 - loss: 1.5891 - val_accuracy: 0.4834 - val_loss: 1.0639
116/116 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.5319 - loss: 0.9836 - val_accuracy: 0.5422 - val_loss: 0.9731
116/116 ━━━━━━━━━━━━━━━━━━━━ 239s 2s/step - accuracy: 0.5741 - loss: 0.9283 - val_accuracy: 0.5989 - val_loss: 0.8805
116/116 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.6200 - loss: 0.8745 - val_accuracy: 0.6834 - val_loss: 0.7218
116/116 ━━━━━━━━━━━━━━━━━━━━ 262s 2s/step - accuracy: 0.6521 - loss: 0.8097 - val_accuracy: 0.6781 - val_loss: 0.7728
116/116 ━━━━━━━━━━━━━━━━━━━━ 233s 2s/step - accuracy: 0.6538 - loss: 0.7790 - val_accuracy: 0.7102 - val_loss: 0.6875
116/116 ━━━━━━━━━━━━━━━━━━━━ 242s 2s/step - accuracy: 0.6512 - loss: 0.7854 - val_accuracy: 0.7102 - val_loss: 0.6864
116/116 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.6869 - loss: 0.7423 - val_accuracy: 0.7070 - val_loss: 0.6788
116/116 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 

In [15]:
#Test Folder Path

test_folder = '/content/drive/My Drive/Chest_Xray/Test_Chest_X_Ray'

In [16]:
# Define preprocessing parameters
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create a test data generator
test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    shuffle=False,
)

Found 634 images belonging to 3 classes.


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Evaluate the model on the test set
test_loss, test_accuracy = X_Ray_Model.evaluate(test_generator)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 119s 6s/step - accuracy: 0.8896 - loss: 0.4513
Test Loss: 0.5505
Test Accuracy: 0.8218


In [19]:
# Save the model
X_Ray_Model.save('/content/drive/My Drive/chest_xray_model.h5')
print("Model saved to drive.")

Model saved to drive.


In [20]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [21]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the saved model
model = tf.keras.models.load_model('/content/drive/My Drive/chest_xray_model.h5')

def predict_image(image):
  image = Image.fromarray(image.astype('uint8'), 'RGB')
  image = image.resize((150, 150))
  image = np.array(image) / 255.0
  image = np.expand_dims(image, axis=0)
  prediction = model.predict(image)
  class_labels = ['Normal', 'Bacterial_Pneumonia', 'Viral_Pneumonia']
  predicted_class = class_labels[np.argmax(prediction)]
  return predicted_class

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Textbox(label="Prediction"),
    title="Chest X-Ray Image Classification",
    description="Upload a chest X-ray image to classify it as Normal, Bacterial Pneumonia, or Viral Pneumonia."
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0427c318cfc2281592.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
